<center> <img src="img/UMD-Wordmark.jpg" alt="Alt text" width="40%"> </center>

### <center>Sustainability Center Training Course</center>

## Welcome

*Explanation on how to use course. Can be video or text*

## Course Objectives
At the completion of this course, the student will be able to:
- (Module #0 - Intro) Define core concepts of Digital Engineering/Manufacturing 
- (Module #1 - Data) Perform data filtering and cleaning using pre-configured digital tools (Python scripts)
- (Module #2 - Model Based Definition) Interact with a model based definition of an engineering component
- (Module #3 - Model Based Enterprise) Incorporate a model based definition of an engineering component into existing business practices
- (Module #4 - Modeling and Simulation) Understand the landscape of incumbent and emerging computational tools. (Evaluate data from modeling to make decisions - For discussion) 
- (Module #5 - Model Based Process Control) Understand the connection from process data collection and process modeling to improved process performance
- (Module #6 - Digital Twins) Understand what is and is not a digital twin. Evaluate the utility of a specific digital twin
- (Module #7 - Digital Engineering Practice) Identify the elements of digital engineering in engineering practice
- (Module #8 - Digital Engineering Challenges) Student will be equipped to go forward and use the knowledge they learned to tackle new challanges

## Modules
**[Module #0 - Intro to Digital Engineering](introToDE.ipynb)**

**[Module #1 - Data](Modules/Module1.ipynb)**
  * [Polymer 3D Printer](Modules/3.%20Advanced%20Topics/3.1.%20Aerotech%20Data%20Collection%20Platform/Aerotech%20Data%20Collection.ipynb)

**[Module #2 - Model Based Definition](Modules/Module2.ipynb)**
  * []()

**[Module #3 - Model Based Enterprise/Engineering (MBE)](Modules/Module3.ipynb)**
  * [3D Printing Cost Estimate](Modules/2.%20Learning%20Modules/2.3.%20Cost%20Estimation%20Analysis/Cost%20Estimation%20using%20GCode.ipynb)

**[Module #4 - Modeling and Simulation](Modules/Module4.ipynb)**
  * [Accelerated Finite Element with GNN](Modules/3.%20Advanced%20Topics/3.2.%20Material%20Properties%20Finite%20Element%20Analysis/Material%20Properties%20FEA.ipynb)

**[Module #5 - Model Based Process Control](Modules/Module5.ipynb)**
  * [Model Predictive Control](Modules/2.%20Learning%20Modules/2.6.%20Model%20Predictive%20Control/Model%20Predictive%20Control.ipynb)

**[Module #6 - Digital Twins](Modules/Module6.ipynb)**
  * [Digital Twin for Smart Scanning](Modules/2.%20Learning%20Modules/2.8.%20Digital%20Twin%20for%20Smart%20Scanning/Digital%20Twin%20for%20Smart%20Scanning.ipynb)

**[Module #7 - Digital Engineering Practice](Modules/Module7.ipynb)**
  * [Sensor Fusion Processing](Modules/3.%20Advanced%20Topics/3.3.%20Sensor%20Fusion%20Processing/Sensor%20Fusion%20Processing.ipynb)
  * [Predictive Maintenance Module](Modules/2.%20Learning%20Modules/2.2.%20Predictive%20Maintenance%20Module/Predictive%20Maintenance%20Module.ipynb)

**[Module #8 - Digital Engineering Challenges](Modules/Module8.ipynb)**

**[Module Index](Modules/moduleIndex.ipynb)**